# Semana 9 - Hipótesis 3 Feature Store

En este notebook busca fabricar un feature store para el posterior entrenamiento usando MLFlow. Este FS va estar basado en un tablón mejorado de features que se cuelga de la hipótesis 3:

**Hipotesis 3**

*Buscando mantener la estrategia de entrenar un modelo global y no multiples modelos por jerarquía, se va transformar el tablón de entrenamiento y generar features que permitan capturar las relaciones stores-familias al convertir estas caracteristicas en variables dummies. Se mejorará la determinación de feriados que se hizo parcialmente en H1 y H2, y se dummificarán también variables de fechas y carácterísticas de tiendas, dado que antes se entrenaron muchas de estas variables como enteros continuos. Con todo, se espera que la fabricación de features nuevas y el poder de MLFlow permita mejorar el performance de H2.*

La sección 3 es la mas relevante y fabrica funciones estandar para la fabricación de features provenientes de las tablas de feriados (`holidays`), precios de combustibles (`oil`) y caracteristicas de tiendas (`stores`). La funcion `make_features()` toma todo lo anterior y fabrica todos las nuevas variables. 

En la sección 4 fabricamos el feature store, deje dos opciones, guarde el primer feature store sin dummificación y el segundo dummificado. El segundo tiene un tamaño final de 194 columnas. 

#### 1. Seteando entorno

In [3]:
from helper import *
librerias = ['numpy', 'pandas', 'matplotlib', 'pathlib', 'seaborn', 'sklearn', 'statsmodels']
load_install_package(librerias)

numpy está instalada y lista para usar.
pandas está instalada y lista para usar.
matplotlib está instalada y lista para usar.
pathlib está instalada y lista para usar.
seaborn está instalada y lista para usar.
sklearn está instalada y lista para usar.
statsmodels está instalada y lista para usar.


In [4]:
# Cargamos las librerias
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
# from pyspark.sql.functions import col, to_timestamp

#### 2. Datos competencia kaggle

In [5]:
#oil=spark.sql('select * from analytics_inversiones.kgl_oil').toPandas()
#train=spark.sql('select * from analytics_inversiones.kgl_train').toPandas()
#test=spark.sql('select * from analytics_inversiones.kgl_test').toPandas()
#holidays=spark.sql('select * from analytics_inversiones.kgl_holidays_events').toPandas()
#stores= spark.sql('select * from analytics_inversiones.kgl_stores').toPandas()
#transactions = spark.sql('select * from analytics_inversiones.kgl_transactions').toPandas()

oil=pd.read_csv('Data/oil.csv')
train=pd.read_csv('Data/train.csv')
test=pd.read_csv('Data/test.csv')
holidays=pd.read_csv('Data/holidays_events.csv')
stores=pd.read_csv('Data/stores.csv')
transactions=pd.read_csv('Data/transactions.csv')

display(train)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


### 3. Feature engineering

Vamos a preparar una serie de features mejoradas a las que hicimos en las hipótesis 1 y 2, partiendo para la asignación correcta de los feriados.

##### 3.1 Feriados

In [6]:
# Preparando features de feriados, generando tablon con feriados locales, regionales y nacionales
# función que arma el tablon
def get_holidays_feat(df_holidays, stores_df):
  strs = stores_df.copy()
  
  # Genero un tablon Date, store y si tiene feriado o no para los feriados locales regionales o nacionales
  local_holidays = holidays.query('type != "Work Day" & locale == "Local"')[['date', 'locale_name']]
  local_holidays.rename(columns={'locale_name': 'city'}, inplace=True)
  region_holidays = holidays.query('type != "Work Day" & locale == "Regional"')[['date', 'locale_name']]
  region_holidays.rename(columns={'locale_name': 'state'}, inplace=True)
  national_holiday = holidays.query('type != "Work Day" & locale == "National"')[['date']]
  national_holiday = national_holiday.drop_duplicates() # eliminamos feriados duplicados

  # Definiendo marcas de feriados
  local_holidays['is_holiday'] = 1
  region_holidays['is_holiday'] = 1
  national_holiday['is_holiday'] = 1
  national_holiday['key'] = 1
  strs['key'] = 1

  # fusionando con las tiendas
  local_holidays = local_holidays.merge(strs, on = 'city', how='left')[['date', 'store_nbr', 'is_holiday']]
  region_holidays = region_holidays.merge(strs, on = 'state', how='left')[['date', 'store_nbr', 'is_holiday']]
  national_holiday = national_holiday.merge(strs, on = 'key', how='left')[['date', 'store_nbr', 'is_holiday']]

  holiday_combined = pd.concat([local_holidays, region_holidays, national_holiday])
  holiday_combined = holiday_combined.drop_duplicates() # eliminamos feriados duplicados al cruzar todo
  holiday_combined['date'] = pd.to_datetime(holiday_combined['date']) # Date a formato fecha
  return holiday_combined

holidays_feat = get_holidays_feat(holidays, stores)
display(holidays_feat)

,date,store_nbr,is_holiday
0,2012-03-02,52,1
1,2012-03-02,53,1
2,2012-04-12,37,1
3,2012-04-12,39,1
4,2012-04-12,42,1
...,...,...,...
8797,2017-12-26,50,1
8798,2017-12-26,51,1
8799,2017-12-26,52,1
8800,2017-12-26,53,1


##### 3.2 Precios Combustible

In [7]:
# Avanzamos con las features del dataset 'oil', haremos el delta de cambios de precio diario, semanal y mensual.
def get_oil_feat(oil_df, start_date, end_date):
  oil_df['date'] = pd.to_datetime(oil['date'])  # dejarlo en fecha
  date_df = pd.DataFrame(pd.date_range(start_date, end_date, freq='D'), columns=['date']) # df vector fechas completo
  merged_df = pd.merge(date_df, oil, on='date', how='left')
  merged_df['dcoilwtico'].fillna(method='ffill', inplace=True)
  merged_df['oil_pct'] = merged_df['dcoilwtico'].pct_change() # delta diario oil
  merged_df['oil_pct7'] = merged_df['dcoilwtico'].pct_change(7) # delta semanal oil
  merged_df['oil_pct30'] = merged_df['dcoilwtico'].pct_change(30) # delta mensual oil
  merged_df.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
  return(merged_df)

start_date = train['date'].min() # inicio fecha presente en el train
end_date = test['date'].max() # ultima fecha presente en el test
oil_feat = get_oil_feat(oil, start_date, end_date)
display(oil_feat)

,date,oil_price,oil_pct,oil_pct7,oil_pct30
0,2013-01-01,NaN,NaN,NaN,NaN
1,2013-01-02,93.14,NaN,NaN,NaN
2,2013-01-03,92.97,-0.001825,NaN,NaN
3,2013-01-04,93.12,0.001613,NaN,NaN
4,2013-01-05,93.12,0.000000,NaN,NaN
...,...,...,...,...,...
1699,2017-08-27,47.65,0.000000,-0.019346,-0.041633
1700,2017-08-28,46.40,-0.026233,-0.020890,-0.066774
1701,2017-08-29,46.46,0.001293,-0.024974,-0.065567
1702,2017-08-30,45.96,-0.010762,-0.051393,-0.084644


##### 3.3 Tiendas

In [8]:
def remove_words(text, remove):
    for word in remove:
        text = text.replace(word, '')
    return text

def get_stores_feat(stores_df):
  words_to_remove = ["el", "los", "de"]
  stores_df['city'] = stores_df['city'].str.lower().apply(remove_words, args=(words_to_remove,))
  stores_df['state'] = stores_df['state'].str.lower().apply(remove_words, args=(words_to_remove,))

  stores_df['city'] = stores_df['city'].str.slice(0, 4).str.strip()
  stores_df['state'] = stores_df['state'].str.slice(0, 4).str.strip()
  return stores_df

stores_feat = get_stores_feat(stores)
display(stores_feat)

,store_nbr,city,state,type,cluster
0,1,quit,pich,D,13
1,2,quit,pich,D,13
2,3,quit,pich,D,8
3,4,quit,pich,D,9
4,5,sant,sant,D,4
5,6,quit,pich,D,13
6,7,quit,pich,D,8
7,8,quit,pich,D,8
8,9,quit,pich,B,6
9,10,quit,pich,C,15


#### 3.4 Features

In [9]:
# Armamos una función que nos genera:
# Features de fecha: basado en mes, dia, semana, fin de semanas, etc
# Features de precios de combustibles (oil_feat)
# Feature de feriados (holidays_feat)
# Features de características de tiendas (stores_feat)
def make_features(df, oil_f, holiday_f, store_f):
  # features date
  df['date'] = pd.to_datetime(df['date']) # Date a formato fecha
  df['month'] = df['date'].dt.month
  df['day'] = df['date'].dt.day
  df['day_of_week'] = df['date'].dt.dayofweek
  df['week_of_month'] = df['date'].apply(week_of_month)
  df['quarter'] = df['date'].dt.quarter
  df['is_weekend'] = (df['date'].dt.weekday >= 5).astype('int')
  df['onpromotion'] = np.log10(df['onpromotion'] + 1) # normalizamos esta variable
  
  # Transformamos la target si está
  if 'sales' in df.columns:
    df['sales'] = np.log10(df['sales'] + 1) 
    
  # features oil
  df = df.merge(oil_feat, on='date', how='left')

  # features holidays
  df = df.merge(holidays_feat, on=['date', 'store_nbr'], how='left')
  df['is_holiday'] = df['is_holiday'].fillna(0) # dejar en cero los dias no feriados

  # features stores
  df = df.merge(stores_feat, on='store_nbr', how='left')

  # Eliminamos filas con vacios (se elimina el primer mes de datos)
  df.dropna(inplace=True)

  return df

train_feat = make_features(train, oil_feat, holidays_feat, stores_feat)
test_feat = make_features(test, oil_feat, holidays_feat, stores_feat)
train_feat['slice'] = 'train'
test_feat['slice'] = 'test'

# Fusiono ambas tablas ya que será un feature store
tbl_feat = pd.concat([train_feat, test_feat])
#tbl_feat = tbl_feat.drop("sales", axis=1) # sacamos la target que no va en el feature store

print("Tablon Train", train_feat.shape)
print("Tablon Test", test_feat.shape)
print("Tablon fusión", tbl_feat.shape)

display(tbl_feat.sort_values(by=['date'], ascending=False))

Tablon Train (2945646, 22)
Tablon Test (28512, 21)
Tablon fusión (2974158, 22)


,id,date,store_nbr,family,sales,onpromotion,month,day,day_of_week,week_of_month,...,oil_price,oil_pct,oil_pct7,oil_pct30,is_holiday,city,state,type,cluster,slice
28511,3029399,2017-08-31,9,SEAFOOD,NaN,0.000000,8,31,3,5,...,47.26,0.028285,0.000423,-0.039236,0.0,quit,pich,B,6,test
27327,3028215,2017-08-31,26,BEVERAGES,NaN,1.518514,8,31,3,5,...,47.26,0.028285,0.000423,-0.039236,0.0,guay,guay,D,10,test
27316,3028204,2017-08-31,25,PERSONAL CARE,NaN,0.845098,8,31,3,5,...,47.26,0.028285,0.000423,-0.039236,0.0,sali,sant,D,1,test
27317,3028205,2017-08-31,25,PET SUPPLIES,NaN,0.000000,8,31,3,5,...,47.26,0.028285,0.000423,-0.039236,0.0,sali,sant,D,1,test
27318,3028206,2017-08-31,25,PLAYERS AND ELECTRONICS,NaN,0.000000,8,31,3,5,...,47.26,0.028285,0.000423,-0.039236,0.0,sali,sant,D,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56434,56434,2013-02-01,42,BOOKS,0.00000,0.000000,2,1,4,1,...,97.46,-0.001946,0.024277,0.046382,0.0,cuen,azua,D,2,train
56435,56435,2013-02-01,42,BREAD/BAKERY,0.00000,0.000000,2,1,4,1,...,97.46,-0.001946,0.024277,0.046382,0.0,cuen,azua,D,2,train
56436,56436,2013-02-01,42,CELEBRATION,0.00000,0.000000,2,1,4,1,...,97.46,-0.001946,0.024277,0.046382,0.0,cuen,azua,D,2,train
56437,56437,2013-02-01,42,CLEANING,0.00000,0.000000,2,1,4,1,...,97.46,-0.001946,0.024277,0.046382,0.0,cuen,azua,D,2,train


In [10]:
def make_df_dummy(df): 
  # Transformamos unas columnas en string
  df = df.astype({
      'store_nbr': 'str',
      'day_of_week': 'str',
      'week_of_month': 'str',
      'quarter': 'str',
      'month': 'str',
      'cluster': 'str'
  })

  # Dummify!
  words_to_remove = [' ', '/', ',']
  df['family'] = df['family'].apply(remove_words, args=(words_to_remove,)) # arreglamos el texto que después pasa a columnas
  df_encoded = pd.get_dummies(df, columns=['day', 'day_of_week', 'week_of_month', 'quarter', 'month', 'city', 'state', 'type', 
                                           'cluster', 'store_nbr', 'family'], 
                              prefix = ['day', 'day_week', 'week_month', 'quarter', 'month', 'city', 'st', 
                                        'type', 'clus', 'store', 'f'],
                              drop_first = True)

  return df_encoded

# Aplicamos la transformación
tbl_feat_encoded = make_df_dummy(tbl_feat)

# Para spark DF, necesito que esté todo en int64
cols_to_convert = tbl_feat_encoded.select_dtypes(include=['bool']).columns
tbl_feat_encoded[cols_to_convert] = tbl_feat_encoded[cols_to_convert].astype('int64')

display(tbl_feat_encoded)

,id,date,sales,onpromotion,is_weekend,oil_price,oil_pct,oil_pct7,oil_pct30,is_holiday,...,f_MAGAZINES,f_MEATS,f_PERSONALCARE,f_PETSUPPLIES,f_PLAYERSANDELECTRONICS,f_POULTRY,f_PREPAREDFOODS,f_PRODUCE,f_SCHOOLANDOFFICESUPPLIES,f_SEAFOOD
55242,55242,2013-02-01,0.602060,0.00000,0,97.46,-0.001946,0.024277,0.046382,0.0,...,0,0,0,0,0,0,0,0,0,0
55243,55243,2013-02-01,0.000000,0.00000,0,97.46,-0.001946,0.024277,0.046382,0.0,...,0,0,0,0,0,0,0,0,0,0
55244,55244,2013-02-01,0.000000,0.00000,0,97.46,-0.001946,0.024277,0.046382,0.0,...,0,0,0,0,0,0,0,0,0,0
55245,55245,2013-02-01,2.974051,0.00000,0,97.46,-0.001946,0.024277,0.046382,0.0,...,0,0,0,0,0,0,0,0,0,0
55246,55246,2013-02-01,0.000000,0.00000,0,97.46,-0.001946,0.024277,0.046382,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,NaN,0.30103,0,47.26,0.028285,0.000423,-0.039236,0.0,...,0,0,0,0,0,1,0,0,0,0
28508,3029396,2017-08-31,NaN,0.00000,0,47.26,0.028285,0.000423,-0.039236,0.0,...,0,0,0,0,0,0,1,0,0,0
28509,3029397,2017-08-31,NaN,0.30103,0,47.26,0.028285,0.000423,-0.039236,0.0,...,0,0,0,0,0,0,0,1,0,0
28510,3029398,2017-08-31,NaN,1.00000,0,47.26,0.028285,0.000423,-0.039236,0.0,...,0,0,0,0,0,0,0,0,1,0


### Respaldando tablon feature

In [15]:
tbl_feat_encoded.to_csv('Data/kgl_tbl_dummy.csv')